In [1]:
import pickle
import pandas as pd
import numpy as np
import torch
print(torch.__version__)

%cd ~/openfold
# %rm out/experiments/angletransformer-make-caches-00/*.pkl
# BASEPATH = 'out/experiments/angletransformer-make-caches-10-65kpt1/cpus/'
BASEPATH = "out/experiments/angletransformer-make-caches-30-Valmin/"
%ls -hlt $BASEPATH*.pkl

1.12.1
/net/pulsar/home/koes/jok120/openfold
-rw-r--r-- 1 jok120 koes 134M Aug  5 16:37 out/experiments/angletransformer-make-caches-30-Valmin/angle_transformer_intermediates2_val.pkl
-rw-r--r-- 1 jok120 koes  79M Aug  5 16:37 out/experiments/angletransformer-make-caches-30-Valmin/angle_transformer_intermediates0_val.pkl
-rw-r--r-- 1 jok120 koes  75M Aug  5 16:37 out/experiments/angletransformer-make-caches-30-Valmin/angle_transformer_intermediates1_val.pkl
-rw-r--r-- 1 jok120 koes  52M Aug  5 16:37 out/experiments/angletransformer-make-caches-30-Valmin/angle_transformer_intermediates3_val.pkl


# Load the data

In [7]:
idx = 2
fn = f"{BASEPATH}angle_transformer_intermediates{idx}_val.pkl"

with open(fn, "rb") as f:
    intermediates = pickle.load(f)

In [ ]:
len(intermediates)

In [ ]:
diter = iter(intermediates.items())

In [ ]:
next(diter)

In [ ]:
list(intermediates.keys())

In [ ]:
intermediates[23]

In [ ]:
k, v = next(diter)
k, v.keys()

In [8]:
d = intermediates

Epoch 53: 100%|██████████| 24/24 [00:18<00:00,  1.33it/s, loss=0.369, v_num=21]

In [ ]:
d['AT']['s'][0].shape, d['AT']['s_initial'][0].shape

In [ ]:
d['batch'].keys()

In [ ]:
for k in ['aatype', 'seq_mask', 'chi_mask', 'chi_angles_sin_cos', 'name']:
    if k != 'name':

        print(k, d['batch'][k][0].shape, len(d['batch'][k]))

In [ ]:
162824/20

In [ ]:
8145*8

In [ ]:
d['AT']['s'][0].shape

In [ ]:
print(len(d['AT']['s']), len(d['AT']['s_initial']))

### Things I need based on the below:

For supervised_chi_loss:
* Input
  - AT input, aka s and s_initial 
      - (1 x 256 x 384), (1 x 256 x 384), d['inputs'][0]
* Output
    * batch["aatype"]
    * batch["seq_mask"]
    * batch["chi_mask"]
    * batch["chi_angles_sin_cos"] (GT angles)

 

In [ ]:
import torch
# AT input: s, s_initial (1 x 256 x 384), (1 x 256 x 384)
print(list(map(lambda x: x.shape, d['inputs'][0])))

# AT output: unnormalized_angles, angles (1 x 256 x 7, 2), (1 x 256 x 7, 2)
print(list(map(lambda x: x.shape, d['outputs'][0])))

# Loss_input: (out, batch)
# out: dict: ['msa', 'pair', 'single', 'sm', 'final_atom_positions', 'final_atom_mask', 
# 'final_affine_tensor', 'lddt_logits', 'plddt', 'distogram_logits', 'masked_msa_logits', 
# 'experimentally_resolved_logits', 'violation']
print(d['loss_inputs'][0][0]['sm']["angles"].shape)
print(d['loss_inputs'][0][1]["chi_angles_sin_cos"].shape)

# Loss_output: (cum_loss, loss_dict)
print(d['loss_outputs'][0])





In [ ]:
d['loss_inputs'][0][0].keys()

# Make a simple model

In [4]:
from openfold.model.jk_sidechain_model import AngleTransformer
from openfold.config import config

from openfold.utils.loss import supervised_chi_loss

/net/pulsar/home/koes/jok120/openfold/lib/conda/envs/openfold_venv/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
at = AngleTransformer(
    c_s=384,
    c_hidden=256,
    no_blocks=2,
    no_angles=config.model.structure_module.no_angles,  # 7
    epsilon=config.globals.eps,
    dropout=0.1,
    d_ff=2048,
    no_heads=4,
    activation='relu')


In [ ]:
d[0]['s_initial']

In [ ]:
unnorm_ang, ang = at(d[0]['s'][0].float()[-1], d[0]['s_initial'][0].float())
aatype, seq_mask, chi_mask, chi_angles_sin_cos = d[0]['aatype'], d[0]['seq_mask'], d[0]['chi_mask'], d[0]['chi_angles_sin_cos']

In [ ]:
unnorm_ang.shape, ang.shape

In [ ]:
# Add a dimension at the start of ang
unnorm_ang2 = unnorm_ang.unsqueeze(0)
ang2 = ang.unsqueeze(0)
# unnorm_ang2 = unnorm_ang
# ang2 = ang

In [ ]:
supervised_chi_loss(angles_sin_cos=ang2,
                    unnormalized_angles_sin_cos=unnorm_ang2,
                    aatype=aatype,
                    seq_mask=seq_mask,
                    chi_mask=chi_mask,
                    chi_angles_sin_cos=chi_angles_sin_cos,
                    chi_weight=config.loss.supervised_chi.chi_weight,
                    angle_norm_weight=config.loss.supervised_chi.angle_norm_weight,
                    eps=1e-6)


In [9]:
# Setup a super simple pytorch lightning loop to make predictions and lower the loss

import pytorch_lightning as pl

class ATModuleLit(pl.LightningModule):
    def __init__(self, dataset_dict):
        super().__init__()
        self.at = self._init_angle_transformer()
        self.loss = supervised_chi_loss
        self.dataset_dict = dataset_dict


    def _init_angle_transformer(self):
        at = AngleTransformer(
            c_s=384,
            c_hidden=256,
            no_blocks=2,
            no_angles=config.model.structure_module.no_angles,  # 7
            epsilon=config.globals.eps,
            dropout=0.1,
            d_ff=2048,
            no_heads=4,
            activation='relu')
        return at

    def forward(self, s, s_initial):
        return self.at(s, s_initial)

    def training_step(self, batch, batch_idx):

        s, s_initial = batch['s'][0].float()[-1], batch['s_initial'][0].float()
        unnorm_ang, ang = self(s, s_initial)
        loss = self.loss(angles_sin_cos=ang.unsqueeze(0),
                         unnormalized_angles_sin_cos=unnorm_ang.unsqueeze(0),
                         aatype=batch['aatype'],
                         seq_mask=batch['seq_mask'],
                         chi_mask=batch['chi_mask'],
                         chi_angles_sin_cos=batch['chi_angles_sin_cos'],
                         chi_weight=config.loss.supervised_chi.chi_weight,
                         angle_norm_weight=config.loss.supervised_chi.angle_norm_weight,
                         eps=1e-6)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(ATDataset(self.dataset_dict),
                                             batch_size=1,
                                             shuffle=True,
                                             num_workers=1,
                                             collate_fn=collate_fn)


class ATDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_dict):
        self.dataset_dict = dataset_dict

    def __len__(self):
        return len(self.dataset_dict) -1

    def __getitem__(self, idx):
        return self.dataset_dict[idx]


def collate_fn(batch):
    return batch[0]





In [10]:
# Train the model

at_lit = ATModuleLit(dataset_dict=d)


trainer = pl.Trainer(max_epochs=10000, gpus=1)

trainer.fit(at_lit)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]



  | Name | Type             | Params
------------------------------------------
0 | at   | AngleTransformer | 2.8 M 
------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.323    Total estimated model params size (MB)


Epoch 3272: 100%|██████████| 24/24 [00:00<00:00, 27.33it/s, loss=0.000969, v_num=22]

Epoch 3272: 100%|██████████| 24/24 [00:18<00:00,  1.33it/s, loss=0.000969, v_num=22]

In [ ]:

step = 0

at_lit.cuda()
opt = torch.optim.Adam(at_lit.at.parameters(), lr=1e-4)

for batch in iter(loader):
    opt.zero_grad()
    s, s_initial = batch['AT']['s'].float().cuda()[0], batch['AT']['s_initial'].float().cuda()[0]
    unnorm_ang, ang = at_lit(s, s_initial)
    loss_dict = supervised_chi_loss(angles_sin_cos=ang.unsqueeze(0).float().cuda(),
                        unnormalized_angles_sin_cos=unnorm_ang.unsqueeze(0).float().cuda(),
                        aatype=batch['aatype'][0].cuda(),
                        seq_mask=batch['seq_mask'][0].cuda(),
                        chi_mask=batch['chi_mask'][0].cuda(),
                        chi_angles_sin_cos=batch['chi_angles_sin_cos'][0].float().cuda(),
                        chi_weight=config.loss.supervised_chi.chi_weight,
                        angle_norm_weight=config.loss.supervised_chi.angle_norm_weight,
                        eps=1e-6)
    loss = loss_dict['loss']
    loss.backward()
    opt.step()
    print(loss.item())
    if step > 100:
        break
    step += 1

    

In [ ]:
batch['AT']['s']